In [1]:
import base64
import requests

In [2]:
import torch
def display_vram_stats():
    left, total = torch.cuda.mem_get_info()
    print(f"Used space - {total/1024/1024/1024 - left/1024/1024/1024}GB/{total/1024/1024/1024}GB")
    return left

In [3]:
display_vram_stats()

Used space - 0.27069091796875GB/23.5882568359375GB


25037045760

# Specs: RTX 3090, 24GB VRAM

## Instruct 3B float16

### Before model download


In [104]:
before_space = display_vram_stats()

Used space - 2.59423828125GB/23.5882568359375GB


#### After model download. Specs: RTX 3090, 24GB VRAM

Engine 000: Avg prompt throughput: 7.3 tokens/s, Avg generation throughput: 1.5 tokens/s

In [11]:
after_space = display_vram_stats()

Used space - 21.45489501953125GB/23.5882568359375GB


## Instruct 7B float16 AWQ

## Model Tests

In [4]:
from pydantic import BaseModel
from openai import OpenAI

class Annotation(BaseModel):
    table_name: str
    table_description: str

class PageTablesAnnotations(BaseModel):
    annotations: list[Annotation]

In [5]:
response_format= {
            "type": "json_schema",
            "json_schema": {
                "name": "PageTablesAnnotations",
                "schema": PageTablesAnnotations.model_json_schema(),
            },
        }

In [6]:
response_format

{'type': 'json_schema',
 'json_schema': {'name': 'PageTablesAnnotations',
  'schema': {'$defs': {'Annotation': {'properties': {'table_name': {'title': 'Table Name',
       'type': 'string'},
      'table_description': {'title': 'Table Description', 'type': 'string'}},
     'required': ['table_name', 'table_description'],
     'title': 'Annotation',
     'type': 'object'}},
   'properties': {'annotations': {'items': {'$ref': '#/$defs/Annotation'},
     'title': 'Annotations',
     'type': 'array'}},
   'required': ['annotations'],
   'title': 'PageTablesAnnotations',
   'type': 'object'}}}

In [7]:
# @title inference function with API
def get_response(model, system_prompt, base64_image, query, url="http://localhost:8000/v1/chat/completions", min_pixels=512*28*28, max_pixels=2048*28*28):
    
    messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{str(base64_image)}"}},
                {"type": "text", "text": query}
            ]}
        ]
    
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "model": model,
        "messages": messages,
        #"extra_body": {"guided_json": response_format},
        "response_format": response_format,
        "temperature": 0.0
    }

    response = requests.post(url, headers=headers, json=data)
    
    return response.json()
    

In [8]:
%%time

model = "Qwen/Qwen2.5-VL-32B-Instruct-AWQ"

with open("/workspace/knowledge-base-chat-bot-develop/data/kzsk/images/pages/1.png", "rb") as image_file:
    buffer = image_file.read() 
    encoded_image = base64.b64encode(buffer).decode("utf-8")

system_prompt = """Ты — помощник, отвечающий только на русском языке, специализирующийся на описании таблиц и составлении содержательных названий из научных статей. Создавай подробные подписи к таблицам(3 предложения), которые отражают результаты приведенных в таблицах сравнений. Подписи и названия таблиц должны быть максимально релевантными для поиска ответов. Используй текст статьи для уточнения деталей. Отдай результат в формате JSON-схемы {'annotations': ['table_name': содержательное название таблицы(оптимизированное для поиска ответов на русском языке), 'description': информативное описание таблицы(оптимизированное для поиска ответов на русском языке)]}"""

query = """
Опиши все таблицы на этой странице сверху вниз. Дай им содержательные короткие названия, которые отвечают на вопрос "Что это за таблица?", и информативные описания, которые отвечают на вопрос "О чем эта таблица?". Текст страницы: 'Table  1 THE CHARACTERISTICS OF H L PDMS COPOLYMERS\n\n<!-- image -->\n\n| Sample   | DH /D   | H"     | 25    | Mv   | GPC   | GPC   | GPC   |\n|----------|---------|--------|-------|------|-------|-------|-------|\n|          |         | [%]    | [cSt] |      | Mn    | Mw    | Mw/Mn |\n| HPDMS-1  | 1:8.4   | 0.1393 | 78    | 7000 | 6250  | 11187 | 1.79  |\n| HPDMS-2  | 1:10    | 0.1312 | 63    | 4000 | 3583  | 5750  | 1.6   |\n\nAs determined by modified Zerewitinonoff metode [16];\n\nblgn25\n\nIn  table  1  the  specific  characteristics of  H l PDMS copolymers are presented. The ratio between CH 3 (H) SiO/ (CH 3 ) 2 SiO (D H /D) was determined from the integrals of the corresponding protons signals for the above mentioned groups in the 1 H-NMR spectrum.\n\n## Characteristics for V PDMS and VcPDMS copolymers\n\nl The  characteristics  of  the  V l PDMS and VcPDMS copolymers are presented in table 2. The ratios CH 2 =CHSi/(CH 3 ) 2 SiO- (V/D), calculated according to the quantities of  the  two  components  introduced  in  reaction  are presented.\n\nThe structure of the V l PDMS and VcPDMS copolymers was initially determined by FT-IR spectroscopy.\n\nIn the 1 H-NMR spectrum (fig.4) of VcPDMS copolymer the characteristic peaks of -CH 3 methyl protons linked to Si atom (at 0.1-0.2 ppm) and the peaks characteristic of protons  from the double bond -Si-CH=CH (at 5.7-6.2 ppm) can be observed.\n\nThe presence of vinyl groups from V l PDMS copolymers could not be the evidence of 1 H-NMR technique due to the small number of vinyl groups in the structure of copolymers.\n\nThermo-mechanical characterisation for polysiloxane networks\n\nThe physico-mechanical properties (table 3) of the crosslinked polymers related to the molar ratio of the -SiCH=CH 2 /-Si-H reactive groups were studied.\n\nFigure  1,  that  presents  the  FT-IR  spectrum  of  the polysiloxanes with vinyl functional groups (V l PDMS and VcPDMS), indicates the presence of the characteristic absorption bands: one at 3052 cm -1 attributed to -Si-CHCH 2 group, the second one at 1597 cm -1 characteristic to the C=C bond and another one at1408 cm -1 determined by =CH 2 grouping.\n\nAs  depicted  in  table  3  the  values  of  the  physicomechanical parameters are quite low, but typical for silicon elastomers [18].\n\nThe dependence of the physical properties of the corresponding siloxanes on temperature was studied by differential scanning calorimetry (DSC) (fig. 4).\n\nThe  glass  temperature Tg (around -118°C) and ÄCp obtained were characteristic for an amorphous phase. Around -48°C an endothermic peak associated with the\n\n| Sample   |         | Conversie   | Mv     | GPC    | GPC     | GPC    |\n|----------|---------|-------------|--------|--------|---------|--------|\n|          | initial | [%]         |        | Mn     | Mw      | Mw/ Mn |\n| VPDMS    | 1:70    | 78          | 95.500 | 50.677 | 104.753 | 2.067  |\n| VcPDMS   | 1:8     | 85          | 594    | 1366   | 1853    | 1.357  |\n\nVID CHz =CHSi-/(CHs)SiO- b1gn25 =1+ 0.0123M"5 [17]\n\nTable  2\n\nTHE CHARACTERISTICS OF V l PDMS AND VcPDMS COPOLYMERS'. Не забудь вернуть ответ в виде JSON."""

query = """
Опиши все таблицы на этой странице сверху вниз. Дай им содержательные короткие названия, которые отвечают на вопрос "Что это за таблица?", и подробные описания, которые отвечают на вопрос "О чем эта таблица?". Текст страницы: ' "Таблица 1 ХАРАКТЕРИСТИКИ H L PDMS СОПОЛИМЕРОВ\n\n<!-- изображение -->\n\n| Образец   | DH /D   | H\"     | 25    | Mv   | GPC   | GPC   | GPC   |\n|-----------|---------|--------|-------|------|-------|-------|-------|\n|           |         | [%]    | [cSt] |      | Mn    | Mw    | Mw/Mn |\n| HPDMS-1   | 1:8.4   | 0.1393 | 78    | 7000 | 6250  | 11187 | 1.79  |\n| HPDMS-2   | 1:10    | 0.1312 | 63    | 4000 | 3583  | 5750  | 1.6   |\n\nКак определено модифицированным методом Зеревитинова [16];\n\nblgn25\n\nВ таблице 1 представлены специфические характеристики сополимеров H L PDMS. Соотношение между CH₃(H)SiO и (CH₃)₂SiO (D H /D) было определено на основе интегралов соответствующих сигналов протонов для вышеуказанных групп в спектре ¹H-NMR.\n\n## Характеристики сополимеров V PDMS и VcPDMS\n\nХарактеристики сополимеров V L PDMS и VcPDMS представлены в таблице 2. Соотношения CH₂=CHSi/(CH₃)₂SiO (V/D) рассчитаны на основе количеств двух компонентов, введённых в реакцию.\n\nСтруктура сополимеров V L PDMS и VcPDMS была первоначально определена с помощью FT-IR спектроскопии.\n\nВ спектре ¹H-NMR (рис. 4) сополимера VcPDMS наблюдаются характерные пики метильных протонов -CH₃, связанных с атомом кремния (в области 0.1–0.2 ppm), и пики, характерные для протонов двойной связи -Si-CH=CH (в области 5.7–6.2 ppm).\n\nНаличие винильных групп в сополимерах V L PDMS не может быть подтверждено методом ¹H-NMR из-за малого количества винильных групп в структуре сополимеров.\n\nТермо-механическая характеристика полисилоксановых сетей\n\nФизико-механические свойства (таблица 3) отверждённых полимеров, связанные с мольным соотношением реакционноспособных групп -SiCH=CH₂/-Si-H, были исследованы.\n\nРисунок 1, представляющий FT-IR спектр полисилоксанов с винильными функциональными группами (V L PDMS и VcPDMS), показывает наличие характерных полос поглощения: одна на 3052 см⁻¹, относящаяся к группе -Si-CHCH₂, вторая на 1597 см⁻¹ — к двойной связи C=C, и ещё одна на 1408 см⁻¹ — к группе =CH₂.\n\nКак показано в таблице 3, значения физико-механических параметров довольно низкие, но типичны для кремнийорганических эластомеров [18].\n\nЗависимость физических свойств соответствующих силоксанов от температуры изучалась методом дифференциальной сканирующей калориметрии (DSC) (рис. 4).\n\nТемпература стеклования Tg (около -118°C) и изменение теплоёмкости ΔCp были характерны для аморфной фазы. В районе -48°C наблюдался эндотермический пик, связанный с\n\n| Образец   |         | Конверсия | Mv     | GPC    | GPC     | GPC    |\n|-----------|---------|-----------|--------|--------|---------|--------|\n|           | нач.    | [%]       |        | Mn     | Mw      | Mw/Mn  |\n| VPDMS     | 1:70    | 78        | 95.500 | 50.677 | 104.753 | 2.067  |\n| VcPDMS    | 1:8     | 85        | 594    | 1366   | 1853    | 1.357  |\n\nVID CH₂=CHSi/(CH₃)SiO- blgn25 = 1 + 0.0123M⁵ [17]\n\nТаблица 2\n\nХАРАКТЕРИСТИКИ СОПОЛИМЕРОВ V L PDMS И VcPDMS. Не забудь вернуть ответ в виде JSON."""


response = get_response(model, system_prompt, encoded_image, query, url="http://localhost:8000/v1/chat/completions")
print(response['choices'][0]['message']['content'])

import json

data = json.loads(response['choices'][0]['message']['content'])
print(data)

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f5ac925e920>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [125]:
%%time

model = "Qwen/Qwen2.5-VL-7B-Instruct-AWQ"

with open("/workspace/knowledge-base-chat-bot-develop/data/kzsk/images/pages/1.png", "rb") as image_file:
    buffer = image_file.read() 
    encoded_image = base64.b64encode(buffer).decode("utf-8")

system_prompt = """Ты — помощник, отвечающий только на русском языке, специализирующийся на описании таблиц и составлении содержательных названий из научных статей. Создавай подробные подписи к таблицам(3 предложения), которые отражают результаты приведенных в таблицах сравнений. Подписи и названия таблиц должны быть максимально релевантными для поиска ответов. Используй текст статьи для уточнения деталей. Отдай результат в формате JSON-схемы {'annotations': ['table_name': содержательное название таблицы(оптимизированное для поиска ответов на русском языке), 'description': информативное описание таблицы(оптимизированное для поиска ответов на русском языке)]}"""

query = """
Опиши все таблицы на этой странице сверху вниз. Дай им содержательные короткие названия, которые отвечают на вопрос "Что это за таблица?", и информативные описания, которые отвечают на вопрос "О чем эта таблица?". Текст страницы: 'Table  1 THE CHARACTERISTICS OF H L PDMS COPOLYMERS\n\n<!-- image -->\n\n| Sample   | DH /D   | H"     | 25    | Mv   | GPC   | GPC   | GPC   |\n|----------|---------|--------|-------|------|-------|-------|-------|\n|          |         | [%]    | [cSt] |      | Mn    | Mw    | Mw/Mn |\n| HPDMS-1  | 1:8.4   | 0.1393 | 78    | 7000 | 6250  | 11187 | 1.79  |\n| HPDMS-2  | 1:10    | 0.1312 | 63    | 4000 | 3583  | 5750  | 1.6   |\n\nAs determined by modified Zerewitinonoff metode [16];\n\nblgn25\n\nIn  table  1  the  specific  characteristics of  H l PDMS copolymers are presented. The ratio between CH 3 (H) SiO/ (CH 3 ) 2 SiO (D H /D) was determined from the integrals of the corresponding protons signals for the above mentioned groups in the 1 H-NMR spectrum.\n\n## Characteristics for V PDMS and VcPDMS copolymers\n\nl The  characteristics  of  the  V l PDMS and VcPDMS copolymers are presented in table 2. The ratios CH 2 =CHSi/(CH 3 ) 2 SiO- (V/D), calculated according to the quantities of  the  two  components  introduced  in  reaction  are presented.\n\nThe structure of the V l PDMS and VcPDMS copolymers was initially determined by FT-IR spectroscopy.\n\nIn the 1 H-NMR spectrum (fig.4) of VcPDMS copolymer the characteristic peaks of -CH 3 methyl protons linked to Si atom (at 0.1-0.2 ppm) and the peaks characteristic of protons  from the double bond -Si-CH=CH (at 5.7-6.2 ppm) can be observed.\n\nThe presence of vinyl groups from V l PDMS copolymers could not be the evidence of 1 H-NMR technique due to the small number of vinyl groups in the structure of copolymers.\n\nThermo-mechanical characterisation for polysiloxane networks\n\nThe physico-mechanical properties (table 3) of the crosslinked polymers related to the molar ratio of the -SiCH=CH 2 /-Si-H reactive groups were studied.\n\nFigure  1,  that  presents  the  FT-IR  spectrum  of  the polysiloxanes with vinyl functional groups (V l PDMS and VcPDMS), indicates the presence of the characteristic absorption bands: one at 3052 cm -1 attributed to -Si-CHCH 2 group, the second one at 1597 cm -1 characteristic to the C=C bond and another one at1408 cm -1 determined by =CH 2 grouping.\n\nAs  depicted  in  table  3  the  values  of  the  physicomechanical parameters are quite low, but typical for silicon elastomers [18].\n\nThe dependence of the physical properties of the corresponding siloxanes on temperature was studied by differential scanning calorimetry (DSC) (fig. 4).\n\nThe  glass  temperature Tg (around -118°C) and ÄCp obtained were characteristic for an amorphous phase. Around -48°C an endothermic peak associated with the\n\n| Sample   |         | Conversie   | Mv     | GPC    | GPC     | GPC    |\n|----------|---------|-------------|--------|--------|---------|--------|\n|          | initial | [%]         |        | Mn     | Mw      | Mw/ Mn |\n| VPDMS    | 1:70    | 78          | 95.500 | 50.677 | 104.753 | 2.067  |\n| VcPDMS   | 1:8     | 85          | 594    | 1366   | 1853    | 1.357  |\n\nVID CHz =CHSi-/(CHs)SiO- b1gn25 =1+ 0.0123M"5 [17]\n\nTable  2\n\nTHE CHARACTERISTICS OF V l PDMS AND VcPDMS COPOLYMERS'. Не забудь вернуть ответ в виде JSON."""

query = """
Опиши все таблицы на этой странице сверху вниз. Дай им содержательные короткие названия, которые отвечают на вопрос "Что это за таблица?", и подробные описания, которые отвечают на вопрос "О чем эта таблица?". Текст страницы: ' "Таблица 1 ХАРАКТЕРИСТИКИ H L PDMS СОПОЛИМЕРОВ\n\n<!-- изображение -->\n\n| Образец   | DH /D   | H\"     | 25    | Mv   | GPC   | GPC   | GPC   |\n|-----------|---------|--------|-------|------|-------|-------|-------|\n|           |         | [%]    | [cSt] |      | Mn    | Mw    | Mw/Mn |\n| HPDMS-1   | 1:8.4   | 0.1393 | 78    | 7000 | 6250  | 11187 | 1.79  |\n| HPDMS-2   | 1:10    | 0.1312 | 63    | 4000 | 3583  | 5750  | 1.6   |\n\nКак определено модифицированным методом Зеревитинова [16];\n\nblgn25\n\nВ таблице 1 представлены специфические характеристики сополимеров H L PDMS. Соотношение между CH₃(H)SiO и (CH₃)₂SiO (D H /D) было определено на основе интегралов соответствующих сигналов протонов для вышеуказанных групп в спектре ¹H-NMR.\n\n## Характеристики сополимеров V PDMS и VcPDMS\n\nХарактеристики сополимеров V L PDMS и VcPDMS представлены в таблице 2. Соотношения CH₂=CHSi/(CH₃)₂SiO (V/D) рассчитаны на основе количеств двух компонентов, введённых в реакцию.\n\nСтруктура сополимеров V L PDMS и VcPDMS была первоначально определена с помощью FT-IR спектроскопии.\n\nВ спектре ¹H-NMR (рис. 4) сополимера VcPDMS наблюдаются характерные пики метильных протонов -CH₃, связанных с атомом кремния (в области 0.1–0.2 ppm), и пики, характерные для протонов двойной связи -Si-CH=CH (в области 5.7–6.2 ppm).\n\nНаличие винильных групп в сополимерах V L PDMS не может быть подтверждено методом ¹H-NMR из-за малого количества винильных групп в структуре сополимеров.\n\nТермо-механическая характеристика полисилоксановых сетей\n\nФизико-механические свойства (таблица 3) отверждённых полимеров, связанные с мольным соотношением реакционноспособных групп -SiCH=CH₂/-Si-H, были исследованы.\n\nРисунок 1, представляющий FT-IR спектр полисилоксанов с винильными функциональными группами (V L PDMS и VcPDMS), показывает наличие характерных полос поглощения: одна на 3052 см⁻¹, относящаяся к группе -Si-CHCH₂, вторая на 1597 см⁻¹ — к двойной связи C=C, и ещё одна на 1408 см⁻¹ — к группе =CH₂.\n\nКак показано в таблице 3, значения физико-механических параметров довольно низкие, но типичны для кремнийорганических эластомеров [18].\n\nЗависимость физических свойств соответствующих силоксанов от температуры изучалась методом дифференциальной сканирующей калориметрии (DSC) (рис. 4).\n\nТемпература стеклования Tg (около -118°C) и изменение теплоёмкости ΔCp были характерны для аморфной фазы. В районе -48°C наблюдался эндотермический пик, связанный с\n\n| Образец   |         | Конверсия | Mv     | GPC    | GPC     | GPC    |\n|-----------|---------|-----------|--------|--------|---------|--------|\n|           | нач.    | [%]       |        | Mn     | Mw      | Mw/Mn  |\n| VPDMS     | 1:70    | 78        | 95.500 | 50.677 | 104.753 | 2.067  |\n| VcPDMS    | 1:8     | 85        | 594    | 1366   | 1853    | 1.357  |\n\nVID CH₂=CHSi/(CH₃)SiO- blgn25 = 1 + 0.0123M⁵ [17]\n\nТаблица 2\n\nХАРАКТЕРИСТИКИ СОПОЛИМЕРОВ V L PDMS И VcPDMS. Не забудь вернуть ответ в виде JSON."""


response = get_response(model, system_prompt, encoded_image, query, url="http://localhost:8000/v1/chat/completions")
print(response)

{'id': 'chatcmpl-83dfadac3cdc4f0d9d2d278aff87c89a', 'object': 'chat.completion', 'created': 1751168836, 'model': 'Qwen/Qwen2.5-VL-7B-Instruct-AWQ', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'reasoning_content': None, 'content': '{\n  "annotations": [\n    {\n      "table_name": "ХАРАКТЕРИСТИКИ H L PDMS СОПОЛИМЕРОВ",\n      "table_description": "Таблица содержит данные о специфических характеристиках сополимеров H L PDMS, включая соотношение между CH₃(H)SiO и (CH₃)₂SiO (DH/D), интегралы соответствующих сигналов протонов и другие физические параметры."\n    },\n    {\n      "table_name": "ХАРАКТЕРИСТИКИ V PDMS И VcPDMS",\n      "table_description": "Таблица 2 предоставляет характеристики сополимеров V PDMS и VcPDMS, включая соотношение CH₂=CHSi/(CH₃)₂SiO (V/D), конверсию и физические параметры."\n    }\n  ]\n}', 'tool_calls': []}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': None}], 'usage': {'prompt_tokens': 2174, 'total_tokens': 2390, 'completion_tokens': 2

In [124]:
print(response['choices'][0]['message']['content'])

import json

data = json.loads(response['choices'][0]['message']['content'])
print(data)

{
  "annotations": [
    {
      "table_name": "Характеристики сополимеров H L PDMS",
      "table_description": "Таблица содержит данные о соотношениях между CH₃(H)SiO и (CH₃)₂SiO (DH/D) для сополимеров H L PDMS, определенные на основе интегралов соответствующих сигналов протонов в спектре ¹H-NMR."
    },
    {
      "table_name": "Характеристики сополимеров V PDMS и VcPDMS",
      "table_description": "Таблица 2 представляет соотношения CH₂=CHSi/(CH₃)₂SiO (V/D) для сополимеров V PDMS и VcPDMS, рассчитанные на основе количеств двух компонентов, введенных в реакцию."
    }
  ]
}
{'annotations': [{'table_name': 'Характеристики сополимеров H L PDMS', 'table_description': 'Таблица содержит данные о соотношениях между CH₃(H)SiO и (CH₃)₂SiO (DH/D) для сополимеров H L PDMS, определенные на основе интегралов соответствующих сигналов протонов в спектре ¹H-NMR.'}, {'table_name': 'Характеристики сополимеров V PDMS и VcPDMS', 'table_description': 'Таблица 2 представляет соотношения CH₂=CHSi/(CH₃)

### ВОпросы:
- «Какие образцы H L PDMS представлены в таблице и каковы их характеристики по GPC?»

«Каким методом были определены соотношения мономеров в H PDMS?»

«Что такое V/D и какие значения указаны для сополимеров V PDMS и VcPDMS?»

In [111]:
%%time

model = "Qwen/Qwen2.5-VL-7B-Instruct-AWQ"

with open("/workspace/knowledge-base-chat-bot-develop/data/kzsk/images/pages/1.png", "rb") as image_file:
    buffer = image_file.read() 
    encoded_image = base64.b64encode(buffer).decode("utf-8")

system_prompt = "Ты — помощник, отвечающий только на русском языке, специализирующийся на описании таблиц и составлении содержательных названий из научных статей. Давай подробные подписи к таблицам, отражающие ключевую информацию, опираясь на предоставленный текст и картинку. Чётко опиши их содержание по 3-4 предложения и названия отражающее назначения таблиц. Твоя подпись должна быть оптимизирована для последующего поиска, чтобы люди могли легко находить ответы на вопросы по этому содержимому. Опирайся на предоставленный текст, помимо картинки - на нем текст со страницы. Отдай ответ в виде JSON-схемы формата - {'annotations': ['table_name': название_таблицы, 'description': описание_таблицы_из_2-3_предложений]}"

query = """
Опиши все таблицы на этой странице сверху вниз. Дай им содержательные короткие названия и информативное описание состоящее из 3 предложений. Опирайся на текст страницы - 'Table  1 THE CHARACTERISTICS OF H L PDMS COPOLYMERS\n\n<!-- image -->\n\n| Sample   | DH /D   | H"     | 25    | Mv   | GPC   | GPC   | GPC   |\n|----------|---------|--------|-------|------|-------|-------|-------|\n|          |         | [%]    | [cSt] |      | Mn    | Mw    | Mw/Mn |\n| HPDMS-1  | 1:8.4   | 0.1393 | 78    | 7000 | 6250  | 11187 | 1.79  |\n| HPDMS-2  | 1:10    | 0.1312 | 63    | 4000 | 3583  | 5750  | 1.6   |\n\nAs determined by modified Zerewitinonoff metode [16];\n\nblgn25\n\nIn  table  1  the  specific  characteristics of  H l PDMS copolymers are presented. The ratio between CH 3 (H) SiO/ (CH 3 ) 2 SiO (D H /D) was determined from the integrals of the corresponding protons signals for the above mentioned groups in the 1 H-NMR spectrum.\n\n## Characteristics for V PDMS and VcPDMS copolymers\n\nl The  characteristics  of  the  V l PDMS and VcPDMS copolymers are presented in table 2. The ratios CH 2 =CHSi/(CH 3 ) 2 SiO- (V/D), calculated according to the quantities of  the  two  components  introduced  in  reaction  are presented.\n\nThe structure of the V l PDMS and VcPDMS copolymers was initially determined by FT-IR spectroscopy.\n\nIn the 1 H-NMR spectrum (fig.4) of VcPDMS copolymer the characteristic peaks of -CH 3 methyl protons linked to Si atom (at 0.1-0.2 ppm) and the peaks characteristic of protons  from the double bond -Si-CH=CH (at 5.7-6.2 ppm) can be observed.\n\nThe presence of vinyl groups from V l PDMS copolymers could not be the evidence of 1 H-NMR technique due to the small number of vinyl groups in the structure of copolymers.\n\nThermo-mechanical characterisation for polysiloxane networks\n\nThe physico-mechanical properties (table 3) of the crosslinked polymers related to the molar ratio of the -SiCH=CH 2 /-Si-H reactive groups were studied.\n\nFigure  1,  that  presents  the  FT-IR  spectrum  of  the polysiloxanes with vinyl functional groups (V l PDMS and VcPDMS), indicates the presence of the characteristic absorption bands: one at 3052 cm -1 attributed to -Si-CHCH 2 group, the second one at 1597 cm -1 characteristic to the C=C bond and another one at1408 cm -1 determined by =CH 2 grouping.\n\nAs  depicted  in  table  3  the  values  of  the  physicomechanical parameters are quite low, but typical for silicon elastomers [18].\n\nThe dependence of the physical properties of the corresponding siloxanes on temperature was studied by differential scanning calorimetry (DSC) (fig. 4).\n\nThe  glass  temperature Tg (around -118°C) and ÄCp obtained were characteristic for an amorphous phase. Around -48°C an endothermic peak associated with the\n\n| Sample   |         | Conversie   | Mv     | GPC    | GPC     | GPC    |\n|----------|---------|-------------|--------|--------|---------|--------|\n|          | initial | [%]         |        | Mn     | Mw      | Mw/ Mn |\n| VPDMS    | 1:70    | 78          | 95.500 | 50.677 | 104.753 | 2.067  |\n| VcPDMS   | 1:8     | 85          | 594    | 1366   | 1853    | 1.357  |\n\nVID CHz =CHSi-/(CHs)SiO- b1gn25 =1+ 0.0123M"5 [17]\n\nTable  2\n\nTHE CHARACTERISTICS OF V l PDMS AND VcPDMS COPOLYMERS'
Не забудь вернуть ответ в виде JSON."""

response = get_response(model, system_prompt, encoded_image, query, url="http://localhost:8000/v1/chat/completions")
print(response)

{'id': 'chatcmpl-2883141cbd8546ababac5aa191bdea94', 'object': 'chat.completion', 'created': 1751165225, 'model': 'Qwen/Qwen2.5-VL-7B-Instruct-AWQ', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'reasoning_content': None, 'content': '{\n  "annotations": [\n    {\n      "table_name": "Table 1",\n      "table_description": "Таблица 1 представляет специфические характеристики полимеров H l PDMS, включая соотношение сигналов протонов CH3 (H) SiO/(CH3)2 SiO (DH/D) в спектре 1Н-NMR."\n    },\n    {\n      "table_name": "Table 2",\n      "table_description": "Таблица 2 показывает характеристики copolимеров V l PDMS и VcPDMS, включая соотношение сигналов протонов CH2=CHSi/(CH3)2 SiO (V/D)."\n    }\n  ]\n}', 'tool_calls': []}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': None}], 'usage': {'prompt_tokens': 1931, 'total_tokens': 2093, 'completion_tokens': 162, 'prompt_tokens_details': None}, 'prompt_logprobs': None, 'kv_transfer_params': None}
CPU times: user 5.84 ms, sys:

In [112]:
print(response['choices'][0]['message']['content'])

import json

data = json.loads(response['choices'][0]['message']['content'])
print(data)

{
  "annotations": [
    {
      "table_name": "Table 1",
      "table_description": "Таблица 1 представляет специфические характеристики полимеров H l PDMS, включая соотношение сигналов протонов CH3 (H) SiO/(CH3)2 SiO (DH/D) в спектре 1Н-NMR."
    },
    {
      "table_name": "Table 2",
      "table_description": "Таблица 2 показывает характеристики copolимеров V l PDMS и VcPDMS, включая соотношение сигналов протонов CH2=CHSi/(CH3)2 SiO (V/D)."
    }
  ]
}
{'annotations': [{'table_name': 'Table 1', 'table_description': 'Таблица 1 представляет специфические характеристики полимеров H l PDMS, включая соотношение сигналов протонов CH3 (H) SiO/(CH3)2 SiO (DH/D) в спектре 1Н-NMR.'}, {'table_name': 'Table 2', 'table_description': 'Таблица 2 показывает характеристики copolимеров V l PDMS и VcPDMS, включая соотношение сигналов протонов CH2=CHSi/(CH3)2 SiO (V/D).'}]}


In [109]:
print(response['choices'][0]['message']['content'])

import json

data = json.loads(response['choices'][0]['message']['content'])
print(data)

{
  "annotations": [
    {
      "table_name": "Table 1",
      "table_description": "Таблица 1 содержит характеристики сополимеров H PDMS, включая соотношение молярных масс и интегральные сигналы протонов в 1Н-NMR спектре."
    },
    {
      "table_name": "Table 2",
      "table_description": "Таблица 2 показывает характеристики сополимеров V PDMS и VcPDMS, включая соотношение молярных масс и интегральные сигналы протонов в 1Н-NMR спектре."
    }
  ]
}
{'annotations': [{'table_name': 'Table 1', 'table_description': 'Таблица 1 содержит характеристики сополимеров H PDMS, включая соотношение молярных масс и интегральные сигналы протонов в 1Н-NMR спектре.'}, {'table_name': 'Table 2', 'table_description': 'Таблица 2 показывает характеристики сополимеров V PDMS и VcPDMS, включая соотношение молярных масс и интегральные сигналы протонов в 1Н-NMR спектре.'}]}


In [98]:
%%time

model = "Qwen/Qwen2.5-VL-3B-Instruct"

with open("/workspace/knowledge-base-chat-bot-develop/data/kzsk/images/pages/1.png", "rb") as image_file:
    buffer = image_file.read() 
    encoded_image = base64.b64encode(buffer).decode("utf-8")

system_prompt = "Вы — помощник, отвечающий только на русском языке, специализирующийся на описании таблиц из научных статей. Давайте подробные подписи к таблицам, отражающие ключевую информацию. Чётко опишите их содержание и назначение. Ваша подпись должна быть оптимизирована для последующего поиска, чтобы люди могли легко находить ответы на вопросы по этому содержимому. Опирайся на предоставленный текст, помимо картинки - на нем текст со страницы. Отдай ответ в виде JSON-схемы формата - {'annotations': ['table_name': название_таблицы, 'description': описание_таблицы_из_2-3_предложений]}"

query = """
Опиши все таблицы на этой странице сверху вниз. Опирайся на текст страницы - 'Table  1 THE CHARACTERISTICS OF H L PDMS COPOLYMERS\n\n<!-- image -->\n\n| Sample   | DH /D   | H"     | 25    | Mv   | GPC   | GPC   | GPC   |\n|----------|---------|--------|-------|------|-------|-------|-------|\n|          |         | [%]    | [cSt] |      | Mn    | Mw    | Mw/Mn |\n| HPDMS-1  | 1:8.4   | 0.1393 | 78    | 7000 | 6250  | 11187 | 1.79  |\n| HPDMS-2  | 1:10    | 0.1312 | 63    | 4000 | 3583  | 5750  | 1.6   |\n\nAs determined by modified Zerewitinonoff metode [16];\n\nblgn25\n\nIn  table  1  the  specific  characteristics of  H l PDMS copolymers are presented. The ratio between CH 3 (H) SiO/ (CH 3 ) 2 SiO (D H /D) was determined from the integrals of the corresponding protons signals for the above mentioned groups in the 1 H-NMR spectrum.\n\n## Characteristics for V PDMS and VcPDMS copolymers\n\nl The  characteristics  of  the  V l PDMS and VcPDMS copolymers are presented in table 2. The ratios CH 2 =CHSi/(CH 3 ) 2 SiO- (V/D), calculated according to the quantities of  the  two  components  introduced  in  reaction  are presented.\n\nThe structure of the V l PDMS and VcPDMS copolymers was initially determined by FT-IR spectroscopy.\n\nIn the 1 H-NMR spectrum (fig.4) of VcPDMS copolymer the characteristic peaks of -CH 3 methyl protons linked to Si atom (at 0.1-0.2 ppm) and the peaks characteristic of protons  from the double bond -Si-CH=CH (at 5.7-6.2 ppm) can be observed.\n\nThe presence of vinyl groups from V l PDMS copolymers could not be the evidence of 1 H-NMR technique due to the small number of vinyl groups in the structure of copolymers.\n\nThermo-mechanical characterisation for polysiloxane networks\n\nThe physico-mechanical properties (table 3) of the crosslinked polymers related to the molar ratio of the -SiCH=CH 2 /-Si-H reactive groups were studied.\n\nFigure  1,  that  presents  the  FT-IR  spectrum  of  the polysiloxanes with vinyl functional groups (V l PDMS and VcPDMS), indicates the presence of the characteristic absorption bands: one at 3052 cm -1 attributed to -Si-CHCH 2 group, the second one at 1597 cm -1 characteristic to the C=C bond and another one at1408 cm -1 determined by =CH 2 grouping.\n\nAs  depicted  in  table  3  the  values  of  the  physicomechanical parameters are quite low, but typical for silicon elastomers [18].\n\nThe dependence of the physical properties of the corresponding siloxanes on temperature was studied by differential scanning calorimetry (DSC) (fig. 4).\n\nThe  glass  temperature Tg (around -118°C) and ÄCp obtained were characteristic for an amorphous phase. Around -48°C an endothermic peak associated with the\n\n| Sample   |         | Conversie   | Mv     | GPC    | GPC     | GPC    |\n|----------|---------|-------------|--------|--------|---------|--------|\n|          | initial | [%]         |        | Mn     | Mw      | Mw/ Mn |\n| VPDMS    | 1:70    | 78          | 95.500 | 50.677 | 104.753 | 2.067  |\n| VcPDMS   | 1:8     | 85          | 594    | 1366   | 1853    | 1.357  |\n\nVID CHz =CHSi-/(CHs)SiO- b1gn25 =1+ 0.0123M"5 [17]\n\nTable  2\n\nTHE CHARACTERISTICS OF V l PDMS AND VcPDMS COPOLYMERS'
Не забудь вернуть ответ в виде JSON."""

response = get_response(model, system_prompt, encoded_image, query, url="http://localhost:8000/v1/chat/completions")
print(response)

{'id': 'chatcmpl-83e16d0c795d470d93254982b452598b', 'object': 'chat.completion', 'created': 1751163321, 'model': 'Qwen/Qwen2.5-VL-3B-Instruct', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'reasoning_content': None, 'content': '{\n  "annotations": [\n    {\n      "table_name": "Table 1",\n      "table_description": "Таблица 1 содержит характеристики полимеров H-PDMS. В ней представлены параметры молекулярной массы и ГПК для двух образцов H-PDMS: HPDMS-1 и HPDMS-2."\n    },\n    {\n      "table_name": "Table 2",\n      "table_description": "Таблица 2 показывает характеристики полимеров VPDMS и VcPDMS. Она включает данные о конверсии, молекулярной массе и ГПК для этих полимеров."\n    }\n  ]\n}', 'tool_calls': []}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': None}], 'usage': {'prompt_tokens': 1902, 'total_tokens': 2058, 'completion_tokens': 156, 'prompt_tokens_details': None}, 'prompt_logprobs': None, 'kv_transfer_params': None}
CPU times: user 5.43 ms, sys: 0 

In [ ]:
"table_name": "Таблица 1",\n      "table_description": "Эта таблица представляет собой характеристики полимеров H l PDMS и V PDMS copolymers. Она показывает отношение CH 3 (H) SiO/ (CH 3 ) 2 SiO (DH /D) и другие физико-механические параметры таких как H"

In [99]:
response['choices'][0]['message']['content']

'{\n  "annotations": [\n    {\n      "table_name": "Table 1",\n      "table_description": "Таблица 1 содержит характеристики полимеров H-PDMS. В ней представлены параметры молекулярной массы и ГПК для двух образцов H-PDMS: HPDMS-1 и HPDMS-2."\n    },\n    {\n      "table_name": "Table 2",\n      "table_description": "Таблица 2 показывает характеристики полимеров VPDMS и VcPDMS. Она включает данные о конверсии, молекулярной массе и ГПК для этих полимеров."\n    }\n  ]\n}'

In [101]:
import json

data = json.loads(response['choices'][0]['message']['content'])
print(data)

{'annotations': [{'table_name': 'Table 1', 'table_description': 'Таблица 1 содержит характеристики полимеров H-PDMS. В ней представлены параметры молекулярной массы и ГПК для двух образцов H-PDMS: HPDMS-1 и HPDMS-2.'}, {'table_name': 'Table 2', 'table_description': 'Таблица 2 показывает характеристики полимеров VPDMS и VcPDMS. Она включает данные о конверсии, молекулярной массе и ГПК для этих полимеров.'}]}


In [75]:
response['choices'][0]['message']['content']

'{\n  "annotations": [\n    {\n      "table_name": "Таблица 3. Свойства резиновых смесей и резин на основе СКЭПТ 60 2 гр. с ЭНБ для производства уплотнителей",\n      "table_description": "Эта таблица представляет собой сравнение различных свойств резиновых смесей, полученных из резины на основе СКЭПТ 60 2 гр. с ЭНБ, при различных условиях. Она включает в себя такие параметры, как вязкость, пластичность, эластическое восстановление, реометрия, крутящий момент, время начала и достижения вулканизации, а также оптимальное время вулканизации. Также представлены данные о вулканизате, включая условную прочность, относительное удлинение при разрыве, твердость по Шору А, сопротивление раздуву и изменения в твердости и прочности после старения."\n    }\n  ]\n}'